# Visualization on Goole Map


In [1]:
import gmaps
# import gmaps.datasets

gmaps.configure(api_key='Key_') # Fill in with your API key

# crime_loc =[]
# for index in range(len(dataset)):
#     crime_loc.append((dataset.at[index,"lat"], dataset.at[index,"lon"]))
#     #     crime_loc = str(dataset.at[index,"lat"])+","+str(dataset.at[index,"lon"])

    
fig = gmaps.figure(map_type='ROADMAP')
# # generate some (latitude, longitude) pairs locations = [(51.5, 0.1), (51.7, 0.2), (51.4, -0.2), (51.49, 0.1)]
# heatmap_layer = gmaps.heatmap_layer(crime_loc)
# fig.add_layer(heatmap_layer) 
fig




Figure(layout=FigureLayout(height='420px'))

# Read KML file (Safe location)

In [86]:
# import urllib, os, json
# import requests
# from IPython.display import Image
import xml.etree.ElementTree as ET
import pandas as pd
import math

In [151]:
# kml_path = "./dataset/safe samples.kml"
kml_path = "./dataset/Safe symbols with light.kml"

In [154]:
tree = ET.parse(kml_path)
root = tree.getroot()
# root
placemarker = tree.findall('.//{http://www.opengis.net/kml/2.2}Placemark')

placemarker[1].text

datalist = []

for attributes in placemarker:
    for subAttribute in attributes:
        if subAttribute.tag == '{http://www.opengis.net/kml/2.2}Point':
            for dsub in subAttribute:
                if dsub.tag == '{http://www.opengis.net/kml/2.2}coordinates':
                    coor = dsub.text
                    coor = coor.replace(",0","")
                    coor = coor.split('\n          ')
                    datalist.append(coor[1].replace(" ",""))
                    break
            break

# len(datalist)
datalist

['-117.0840969,32.5947489',
 '-117.0737825,32.5841174',
 '-117.0651994,32.5840089',
 '-117.0841227,32.5573874',
 '-117.0658407,32.5468248',
 '-117.0498762,32.5657065',
 '-117.0493613,32.5611492',
 '-117.0582018,32.5714932',
 '-117.052537,32.5838609',
 '-117.045437,32.5777776',
 '-117.0454742,32.5813368',
 '-117.0225679,32.5760652',
 '-117.0272304,32.5739431',
 '-117.014256,32.5692968',
 '-117.0334967,32.5640063',
 '-117.0009341,32.583537',
 '-116.99896,32.5758706',
 '-116.9884098,32.5593624',
 '-116.9516743,32.5678259',
 '-116.9532193,32.5533579',
 '-117.1070028,32.7041862',
 '-117.1022821,32.7040598',
 '-117.1098423,32.7054922',
 '-117.1115589,32.7022782',
 '-117.1095633,32.7022601',
 '-117.1121168,32.7048602',
 '-117.114681,32.7042102',
 '-117.112546,32.7037859',
 '-117.1125245,32.7028831',
 '-117.1139461,32.7035918',
 '-117.1168482,32.704147',
 '-117.1157646,32.7035963',
 '-117.1158934,32.7030998',
 '-117.1180606,32.7048963',
 '-117.1187794,32.7038942',
 '-117.1187687,32.7021699',
 

# Output to the csv file

In [171]:
def numberOflights(point_lat,point_lon,distance):
#     R=6378137
#     R=6373000
    R=6372800
    lat,lon = areacalculate(point_lat,point_lon,distance)
    top_right = [lon,lat]

    lat,lon = areacalculate(point_lat,point_lon,-distance)
    bottom_left = [lon,lat]

    lights = all_light_df.loc[(all_light_df['longitude'] <= top_right[0] )& (all_light_df['longitude'] >= bottom_left[0] )\
                     & ( all_light_df['latitude'] >=  bottom_left[1] ) & (all_light_df['latitude'] <= top_right[1]) ]
    
    if len(lights) ==0:
        return 0, 0
    
    dist_list=[]
    for _, alight in lights.iterrows():
        dlon = math.radians(point_lon - alight["longitude"])
        dlat = math.radians(point_lat - alight["latitude"])
        phi1, phi2 = math.radians(point_lat), math.radians(alight["latitude"])
        a = math.sin(dlat/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlon/2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = R * c
        dist_list.append(distance)
        
#     return len(lights), 0
    return len(lights), min(dist_list) 

In [156]:
def areacalculate(lat,lon,offset):
    R=6378137
    dn = offset
    de = offset

    dLat = dn/R
    dLon = de/(R*math.cos(math.pi*lat/180))

    # OffsetPosition, decimal degrees
    latO = lat + dLat * 180/math.pi
    lonO = lon + dLon * 180/math.pi
    return latO,lonO

In [172]:
# Create dataframe for every safe point
dataset_cols =["id",'windows','car','lights','gas',"convience","sdpd",'lampdist','lat','lon','crime']
safe_df = pd.DataFrame(columns=dataset_cols)
for idx, val in enumerate(datalist):
    safe_id = "s_" + str(idx)
    latlon = val.split(",")
    safe_df.loc[idx] = {'id':safe_id , "windows": 0, "car": 0, "lights": 0, "gas": 0, "convience": 0 \
                   ,"sdpd": 0, "lampdist": 0, "lat": latlon[1], "lon": latlon[0], "crime": "safe"}
    
    area_dist = 20
    number, lampdist = numberOflights(float(latlon[1]),float(latlon[0]),distance=area_dist)
    safe_df.at[idx,"lights"]  =number
    while(number == 0):
#         print(number)
        area_dist +=500
        number , lampdist= numberOflights(float(latlon[1]),float(latlon[0]),distance=area_dist)

    safe_df.at[idx,"lampdist"]  =lampdist
#     print(idx)
safe_df

,id,windows,car,lights,gas,convience,sdpd,lampdist,lat,lon,crime
0,s_0,0,0,2,0,0,0,15.4917,32.5947489,-117.0840969,safe
1,s_1,0,0,0,0,0,0,43.8729,32.5841174,-117.0737825,safe
2,s_2,0,0,0,0,0,0,70.8771,32.5840089,-117.0651994,safe
3,s_3,0,0,0,0,0,0,25.5713,32.5573874,-117.0841227,safe
4,s_4,0,0,0,0,0,0,39.3728,32.5468248,-117.0658407,safe
5,s_5,0,0,1,0,0,0,18.176,32.5657065,-117.0498762,safe
6,s_6,0,0,0,0,0,0,53.971,32.5611492,-117.0493613,safe
7,s_7,0,0,0,0,0,0,33.5062,32.5714932,-117.0582018,safe
8,s_8,0,0,0,0,0,0,37.8824,32.5838609,-117.052537,safe
9,s_9,0,0,0,0,0,0,48.8604,32.5777776,-117.045437,safe


In [158]:
#Load light dataframe
light_path = './dataset/streetlight_locations_datasd.csv'
light_cols =['latitude','longitude'] #Columns which need

all_light_df = pd.read_csv(light_path,usecols =light_cols).dropna()
all_light_df.sort_values(by=light_cols, inplace=True, ascending=True)

In [ ]:
for index in range(len(dataset)):
    number = numberOflights(dataset.at[index,"lat"],dataset.at[index,"lon"],distance=20)
    dataset.at[index,"lights"]  =number

In [170]:
# safe_df
safe_df.loc[safe_df["lampdist"] < 50]

,id,windows,car,lights,gas,convience,sdpd,lampdist,lat,lon,crime
